In [2]:
import os
import shutil
import numpy as np
import pickle
import torch
import torch.nn
from torch_geometric.data import Dataset
from torch_geometric.data import Data, HeteroData
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from torch_geometric.nn.conv.gcn_conv import gcn_norm

In [3]:
from collections import defaultdict
from utils import *

In [4]:
%load_ext autoreload
%autoreload 2

In [4]:
data_dir = "all_designs_netlist_data/"
design_fp = "Design_100_netlist_data/"

In [12]:
with open(os.path.join(data_dir, design_fp, 'random.pkl'), 'rb') as f:
    random_dict = pickle.load(f)
    
    node_ramdom_features = random_dict['node_random']
    net_random_features = random_dict['net_random']

In [5]:
from pyg_dataset import NetlistDataset

In [35]:
dataset = NetlistDataset(data_dir="all_designs_netlist_data", load_pe = True, pl = True, processed = True, load_indices=[0, 1, 2])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.87it/s]


In [38]:
all_design_nums = []

for design_fp in os.listdir("target_data"):
    design_num_to_match = design_fp.split("_")[1]
    all_design_nums.append(design_num_to_match)

all_design_nums = np.unique(all_design_nums)

In [40]:
np.save("all_target_design_nums.npy", all_design_nums)

In [42]:
from collections import defaultdict

In [52]:
design_num_dict = defaultdict(list)
design_design_num_dict = defaultdict(int)

In [53]:
target_data_dir = "target_data/"

for fp in os.listdir("all_designs_netlist_data/"):
    design_num = fp.split("_")[1]
    
    with open("all_designs_netlist_data/" + fp + "/node_size_x.pkl", 'rb') as f:
        node_size_x = pickle.load(f)
        
    design_design_num_dict[design_num] = node_size_x.shape[0]

    for design_fp in os.listdir(target_data_dir):
        design_num_to_match = design_fp.split("_")[1]
        if design_num == design_num_to_match:
            with open(os.path.join(target_data_dir, design_fp, 'node_loc_x.pkl'), 'rb') as f:
                node_loc_x = pickle.load(f)
    
            design_num_dict[design_num].append(node_loc_x.shape[0])

In [69]:
for key, value in design_design_num_dict.items():
    if len(design_num_dict[key]) == 0:
        continue
    print(key, design_design_num_dict[key] == design_num_dict[key][0])

195 True
112 True
127 True
145 True
130 True
85 True
207 True
196 True
46 True
240 True
222 True
55 True
82 True
180 True
161 True
52 True
162 True
37 True
21 True
67 True
237 True
115 True
176 True
10 True
221 True
7 True
100 True
147 True


In [37]:
h_dataset = []
for data in tqdm(dataset):    
    out_degrees = data.net_features[:, 1]
    mask = (out_degrees < 3000)
    mask_edges = mask[data.edge_index_source_to_net[1]] 
    filtered_edge_index = data.edge_index_source_to_net[:, mask_edges]
    data.edge_index_source_to_net = filtered_edge_index

    mask_edges = mask[data.edge_index_sink_to_net[1]] 
    filtered_edge_index = data.edge_index_sink_to_net[:, mask_edges]
    data.edge_index_sink_to_net = filtered_edge_index
    
    h_data = HeteroData()
    h_data['node'].x = data.node_features
    h_data['net'].x = data.net_features
    h_data['node', 'as_a_sink_of', 'net'].edge_index, h_data['node', 'as_a_sink_of', 'net'].edge_weight = gcn_norm(data.edge_index_sink_to_net, add_self_loops=False)
    h_data['node', 'as_a_source_of', 'net'].edge_index = data.edge_index_source_to_net
    _, h_data['net', 'sink_to', 'node'].edge_weight = gcn_norm(torch.flip(h_data['node', 'as_a_sink_of', 'net'].edge_index, dims=[0]), add_self_loops=False)
    
    design_num = data['design_name'].split("_")[1]
    variant_data_lst = []
    for design_fp in os.listdir("target_data"):
        design_num_to_match = design_fp.split("_")[1]
        if design_num == design_num_to_match:
            with open(os.path.join("target_data", design_fp, 'node_loc_x.pkl'), 'rb') as f:
                node_loc_x = pickle.load(f)

            with open(os.path.join("target_data", design_fp, 'node_loc_y.pkl'), 'rb') as f:
                node_loc_y = pickle.load(f)

            with open(os.path.join("target_data", design_fp, 'target_net_hpwl.pkl'), 'rb') as f:
                net_hpwl = pickle.load(f)
            
            with open(os.path.join("target_data", design_fp, 'target_node_utilization.pkl'), 'rb') as f:
                node_congestion = pickle.load(f)

            pos_lst = torch.tensor(np.vstack([node_loc_x, node_loc_y]).T)
            node_congestion = torch.tensor(node_congestion).float()
            net_hpwl = torch.tensor(net_hpwl).float()

            node_pos_lst = pos_lst[data.edge_index_sink_to_net[0]]
            net_pos_lst = pos_lst[data.edge_index_source_to_net[0]][data.edge_index_sink_to_net[1]]
            edge_attr = torch.sum(torch.abs(node_pos_lst - net_pos_lst), dim=1)

            file_name = os.path.join("target_data", design_fp, 'pl_part_dict.pkl')
            f = open(file_name, 'rb')
            part_dict = pickle.load(f)
            f.close()
            batch = [part_dict[idx] for idx in range(node_congestion.shape[0])]
            num_vn = len(np.unique(batch))
            batch = torch.tensor(batch).long()
            variant_data_lst.append((pos_lst, edge_attr, node_congestion, net_hpwl, batch, num_vn))

    h_data['variant_data_lst'] = variant_data_lst
    h_dataset.append(h_data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:22<00:00, 67.36s/it]


In [79]:
all_files = np.array(os.listdir(data_dir))
        
if load_indices is not None:
    load_indices = np.array(load_indices)
    all_files = all_files[load_indices]
    
for design_fp in tqdm(all_files):
    data_load_fp = os.path.join(data_dir, design_fp, 'pyg_data.pkl')
    if os.path.exists(data_load_fp):
        data = torch.load(data_load_fp)
    else:
        with open(os.path.join(data_dir, design_fp, 'net2sink_nodes.pkl'), 'rb') as f:
            net2sink = pickle.load(f)

        with open(os.path.join(data_dir, design_fp, 'net2source_node.pkl'), 'rb') as f:
            net2source = pickle.load(f)

        with open(os.path.join(data_dir, design_fp, 'node_type_id.pkl'), 'rb') as f:
            node_type = pickle.load(f)
        
        with open(os.path.join(data_dir, design_fp, 'node_loc_x.pkl'), 'rb') as f:
            node_loc_x = pickle.load(f)
        
        with open(os.path.join(data_dir, design_fp, 'node_loc_y.pkl'), 'rb') as f:
            node_loc_y = pickle.load(f)
        
        with open(os.path.join(data_dir, design_fp, 'node_size_x.pkl'), 'rb') as f:
            node_size_x = pickle.load(f)
        
        with open(os.path.join(data_dir, design_fp, 'node_size_y.pkl'), 'rb') as f:
            node_size_y = pickle.load(f)

        with open(os.path.join(data_dir, design_fp, 'target_net_hpwl.pkl'), 'rb') as f:
            net_hpwl = pickle.load(f)
        
        with open(os.path.join(data_dir, design_fp, 'target_node_congestion_level.pkl'), 'rb') as f:
            node_congestion = pickle.load(f)

        with open(os.path.join(data_dir, design_fp, 'eigen.10.pkl'), 'rb') as f:
            eig_dict = pickle.load(f)
            eig_vec = eig_dict['evects']

        num_instances = len(node_type)
        assert len(node_loc_x) == num_instances
        assert len(node_loc_y) == num_instances
        assert len(node_size_x) == num_instances
        assert len(node_size_y) == num_instances
        assert len(eig_vec) == num_instances
        assert len(node_congestion) == num_instances

        edge_index_source_sink = []
        #edge_index_sink_source = []
        edge_index_sink_to_net = []
        edge_index_source_to_net = []
        
        for net_idx in range(len(net2sink)):
            sink_idx_lst = net2sink[net_idx]
            source_idx = net2source[net_idx]
        
            for sink_idx in sink_idx_lst:
                edge_index_sink_to_net.append([sink_idx, net_idx])
                edge_index_source_sink.append([source_idx, sink_idx])
                #edge_index_sink_source.append([sink_idx, source_idx])
        
            edge_index_source_to_net.append([source_idx, net_idx])
            

        edge_index_source_sink = torch.tensor(edge_index_source_sink).T.long()
        #edge_index_sink_source = torch.tensor(edge_index_sink_source).T.long()
        edge_index_source_to_net = torch.tensor(edge_index_source_to_net).T.long()
        edge_index_sink_to_net = torch.tensor(edge_index_sink_to_net).T.long()
        out_degrees = compute_degrees(edge_index_source_sink, num_instances)
        in_degrees = compute_degrees(torch.flip(edge_index_source_sink, dims=[0]), num_instances)
        source2net_inst_degrees = compute_degrees(edge_index_source_to_net, num_instances)
        #sink2net_inst_degrees = compute_degrees(edge_index_sink_to_net, num_instances)
        source2net_net_degrees = compute_degrees(torch.flip(edge_index_source_to_net, dims=[0]), len(net_hpwl))
        sink2net_net_degrees = compute_degrees(torch.flip(edge_index_sink_to_net, dims=[0]), len(net_hpwl))
                
        if pl:
            node_features = np.vstack([node_type, in_degrees, out_degrees, source2net_inst_degrees, node_size_x, node_size_y, node_loc_x, node_loc_y]).T  
            file_name = os.path.join(data_dir, design_fp, 'pl_part_dict.pkl')
            f = open(file_name, 'rb')
            part_dict = pickle.load(f)
            f.close()
            batch = [part_dict[idx] for idx in range(node_features.shape[0])]
            num_vn = len(np.unique(batch))
            batch = torch.tensor(batch).long()

        else:
            node_features = np.vstack([node_type, in_degrees, out_degrees, source2net_inst_degrees, node_size_x, node_size_y]).T
            batch = None
            num_vn = 0

        if load_pe:
            node_features = np.concatenate([node_features, eig_vec], axis=1)
            
        if density:
            node_route_utilization = np.zeros(num_instances)
            node_pin_utilization = np.zeros(num_instances)
            with open(os.path.join(data_dir, design_fp, "pin_utilization_map.pkl"), 'rb') as f:
                pin_utilization_map = pickle.load(f)
            with open(os.path.join(data_dir, design_fp, "route_utilization_map.pkl"), 'rb') as f:
                route_utilization_map = pickle.load(f)

            for node_idx in range(num_instances):
                node_pin_utilization[node_idx] = pin_utilization_map[int(node_loc_x[node_idx]), int(node_loc_y[node_idx])]
                node_route_utilization[node_idx] = route_utilization_map[int(node_loc_x[node_idx]), int(node_loc_y[node_idx])]

            node_pin_utilization = node_pin_utilization.reshape(num_instances, 1)
            node_route_utilization = node_route_utilization.reshape(num_instances, 1)
            node_features = np.concatenate([node_features, node_pin_utilization, node_route_utilization], axis=1)
                
        node_features = torch.tensor(node_features).float()
        net_features = torch.tensor(np.vstack([source2net_net_degrees, sink2net_net_degrees]).T).float()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.85s/it]


In [10]:
data_dir = "all_designs_netlist_data/"

In [220]:
all_files = np.array(os.listdir(data_dir))

h_dataset = []
    
for design_fp in tqdm(all_files):
    with open(os.path.join(data_dir, design_fp, 'node_loc_x.pkl'), 'rb') as f:
        node_loc_x = pickle.load(f)
    with open(os.path.join(data_dir, design_fp, 'node_loc_y.pkl'), 'rb') as f:
        node_loc_y = pickle.load(f)

    if node_loc_x.shape[0] != 633162:
        continue
        
    with open(os.path.join(data_dir, design_fp, 'target_net_hpwl.pkl'), 'rb') as f:
        net_hpwl = pickle.load(f)
    with open(os.path.join(data_dir, design_fp, 'target_node_congestion_level.pkl'), 'rb') as f:
        node_congestion = pickle.load(f)

    if np.mean(node_congestion) == 0:
        continue

    file_name = os.path.join(data_dir, design_fp, 'pl_part_dict.pkl')
    f = open(file_name, 'rb')
    part_dict = pickle.load(f)
    f.close()
    batch = [part_dict[idx] for idx in range(node_congestion.shape[0])]
    num_vn = len(np.unique(batch))
    batch = torch.tensor(batch).long()

    file_name = os.path.join(data_dir, design_fp, 'top_pl_part_dict.pkl')
    f = open(file_name, 'rb')
    top_part_dict = pickle.load(f)
    f.close()
    top_batch = [top_part_dict[idx] for idx in range(node_congestion.shape[0])]
    num_top_vn = len(np.unique(top_batch))
    top_batch = torch.tensor(top_batch).long()
    
    node_congestion = torch.tensor(node_congestion)
    net_hpwl = torch.tensor(net_hpwl).float()
    
    node_congestion = (node_congestion >= 3).long()
    num_instances = node_congestion.shape[0]
    
    pos_lst = torch.tensor(np.vstack([node_loc_x, node_loc_y]).T)
    
    node_pos_lst = pos_lst[data.edge_index_sink_to_net[0]]
    net_pos_lst = pos_lst[data.edge_index_source_to_net[0]][data.edge_index_sink_to_net[1]]
    
    out_degrees = data.net_features[:, 1]
    mask = (out_degrees < 3000)
    mask_edges = mask[data.edge_index_source_to_net[1]] 
    filtered_edge_index = data.edge_index_source_to_net[:, mask_edges]
    data.edge_index_source_to_net = filtered_edge_index
    
    h_data = HeteroData()
    h_data['node'].x = data.node_features
    h_data['net'].x = data.net_features
    
    h_data['node', 'as_a_sink_of', 'net'].edge_index, h_data['node', 'as_a_sink_of', 'net'].edge_weight = gcn_norm(data.edge_index_sink_to_net, add_self_loops=False)
    h_data['node', 'as_a_source_of', 'net'].edge_index = data.edge_index_source_to_net
    h_data['node', 'as_a_sink_of', 'net'].edge_attr = torch.sum(torch.abs(node_pos_lst - net_pos_lst), dim=1)[mask_edges]
    
    h_data.batch = batch
    h_data.num_vn = num_vn
    h_data.num_instances = num_instances
    
    h_data['node', 'as_a_sink_of', 'net'].edge_index, h_data['node', 'as_a_sink_of', 'net'].edge_weight = gcn_norm(data.edge_index_sink_to_net, add_self_loops=False)
    h_data['node', 'as_a_source_of', 'net'].edge_index = data.edge_index_source_to_net
    _, h_data['net', 'sink_to', 'node'].edge_weight = gcn_norm(torch.flip(h_data['node', 'as_a_sink_of', 'net'].edge_index, dims=[0]), add_self_loops=False)

    h_data.pos = pos_lst
    h_data.num_instances = num_instances
    h_dataset.append(h_data)

 28%|████████████████████████████████████████████████▍                                                                                                                            | 42/150 [00:00<00:01, 93.75it/s]


AttributeError: 'HeteroData' has no attribute 'edge_index_sink_to_net'

In [120]:
design_num = data.design_name.split("_")[1]

In [121]:
torch.save(new_dataset, f"../{design_num}_dataset.pt")

In [18]:
#if load_pe:
node_features = np.concatenate([node_features, eig_vec], axis=1)
        
node_features = torch.tensor(node_features).float()
node_features = torch.concat([node_features, node_ramdom_features], dim=1)
net_features = torch.tensor(np.vstack([source2net_net_degrees, sink2net_net_degrees]).T).float()
net_features = torch.concat([net_features, net_random_features], dim=1)
    
data = Data(
    node_features = node_features, 
    net_features = net_features, 
    edge_index_source_sink = edge_index_source_sink,
    edge_index_sink_to_net = edge_index_sink_to_net, 
    edge_index_source_to_net = edge_index_source_to_net, 
    num_vn = num_vn
)

In [91]:
h_data['node'].x[:, :-2].shape

torch.Size([633162, 19])

In [65]:
h_dataset[0]['net'].y

tensor([1.7800e+02, 1.0355e-01, 7.3000e+01,  ..., 9.7500e+00, 1.0750e+01,
        2.3750e+01])

In [127]:
data_dir = "cross_design_data"
all_files = np.array(os.listdir(data_dir))
load_indices = []
idx = 0
for design_fp in tqdm(all_files):
    if design_fp.split("_")[1] == "107":
        load_indices.append(idx)
    idx += 1

100%|█████████████████████████████████████| 470/470 [00:00<00:00, 641915.62it/s]


In [95]:
data.node_congestion = (data.node_congestion >= 3).long()
data.num_instances = data.node_congestion.shape[0]
out_degrees = data.node_features[:, 2]
mask = (out_degrees < 3000)
new_indices = torch.nonzero(mask, as_tuple=False).view(-1)
mapping = -torch.ones(data.node_features.size(0), dtype=torch.long)
mapping[new_indices] = torch.arange(new_indices.size(0))
# Filter edges where both nodes are in the selected nodes
mask_edges = mask[data.edge_index_source_sink[0]] & mask[data.edge_index_source_sink[1]]
filtered_edge_index = data.edge_index_source_sink[:, mask_edges]

In [142]:
for data in dataset:
    num_instances = data.node_congestion.shape[0]
    data.num_instances = num_instances
    
    data.edge_index_source_to_net[1] = data.edge_index_source_to_net[1] - num_instances
    data.edge_index_sink_to_net[1] = data.edge_index_sink_to_net[1] - num_instances
    
    edge_index_node_to_net = torch.cat([data.edge_index_source_to_net, data.edge_index_sink_to_net], dim=1)
    #data.edge_index_net_to_node = torch.flip(edge_index_node_to_net, [0])
    
    data.edge_index_source_sink = None
    data.edge_index_sink_source = None

In [81]:
h_dataset = torch.load("../h_dataset.pt")

In [119]:
select_num_instances = h_dataset[-7].num_instances

In [120]:
all_indices = []
for idx in range(len(h_dataset)):
    h_data = h_dataset[idx]
    if h_data.num_instances == select_num_instances:
        all_indices.append(idx)

In [177]:
design_indices_dict = defaultdict(list)
for idx in range(len(h_dataset)):
    data = h_dataset[idx]
    design_name = data.design_name
    design_index = int(design_name.split("_")[1])
    design_indices_dict[design_index].append(idx)

AttributeError: 'HeteroData' has no attribute 'design_name'

In [176]:
#start building split index
all_train_indices = []
all_valid_indices = []
all_test_indices = []

for design, design_indices in design_indices_dict.items():
    if len(design_indices) <= 2:
        continue
    train_indices, valid_indices = train_test_split(design_indices, test_size=0.1, random_state=1)
    #test_indices, valid_indices = train_test_split(test_indices, test_size=0.5, random_state=1)
    all_train_indices.append(train_indices)
    all_valid_indices.append(valid_indices)
    #all_test_indices.append(test_indices)

NameError: name 'design_indices_dict' is not defined

In [12]:
all_train_indices = np.concatenate(all_train_indices)
all_valid_indices = np.concatenate(all_valid_indices)
#all_test_indices = np.concatenate(all_test_indices)

In [16]:
pickle.dump((all_train_indices, all_valid_indices, all_test_indices), open("cross_design_data_split.pt", "wb"))

In [122]:
data_dir = "15_designs_regions/"

In [165]:
for design_fp in os.listdir(data_dir):
    fp_dir = data_dir + design_fp + "/"

    design_num = design_fp.split("_")[1]

    node2regionbox = pickle.load(open(fp_dir + "node2regionbox.pkl", "rb"))
    regionbox2xh = pickle.load(open(fp_dir + "regionbox2xh.pkl", "rb"))
    regionbox2xl = pickle.load(open(fp_dir + "regionbox2xl.pkl", "rb"))
    regionbox2yh = pickle.load(open(fp_dir + "regionbox2yh.pkl", "rb"))
    regionbox2yl = pickle.load(open(fp_dir + "regionbox2yl.pkl", "rb"))
    break

In [167]:
node2regionbox += 1

In [168]:
region_box_pos = np.vstack([regionbox2xh, regionbox2xl, regionbox2yh, regionbox2yl]).T

In [169]:
region_box_pos = np.concatenate([-np.ones((1, regionbox2xh.shape[0])), region_box_pos])

In [170]:
node2regionbox.shape[0]

674691

In [175]:
region_features = region_box_pos[node2regionbox]

In [57]:
np.random.shuffle(all_train_indices)

In [66]:
for design_fp in tqdm(os.listdir("cross_design_data/")):
    new_directory_path = f'processed_datasets/{design_fp}'
    os.makedirs(new_directory_path, exist_ok=True)

    source_file_path = f'cross_design_data/{design_fp}/pyg_data.pkl'  
    destination_file_path = os.path.join(new_directory_path, os.path.basename(source_file_path))

    shutil.move(source_file_path, destination_file_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 4591.06it/s]


In [104]:
data = dataset[1]

In [105]:
data

Data(node_features=[600206, 21], net_features=[643597, 12], edge_index_source_sink=[2, 1564639], edge_index_sink_to_net=[2, 1564639], edge_index_source_to_net=[2, 643597], num_vn=0, design_name='Design_136_dmp_1000_data')

In [135]:
from torch_geometric.data import HeteroData

In [136]:
h_data_lst = []
for data in dataset:
    
    h_data = HeteroData()
    h_data['node'].x = data.node_features
    h_data['node'].y = data.node_congestion
    
    h_data['net'].x = data.net_features
    h_data['net'].y = data.net_hpwl
    
    h_data['node', 'as_a_sink_of', 'net'].edge_index = data.edge_index_sink_to_net
    h_data['node', 'as_a_source_of', 'net'].edge_index = data.edge_index_source_to_net
    h_data['net', 'connected_to', 'node'].edge_index = data.edge_index_net_to_node

    h_data.batch = data.batch
    h_data.num_vn = data.num_vn
    h_data_lst.append(h_data)

In [37]:
torch.save(h_data, "h_data.pt")

In [92]:
from torch_geometric.loader import NeighborLoader

In [137]:
l_data = h_data

In [ ]:
loader = NeighborLoader(
                l_data,
                num_neighbors={key: [10] * 4 for key in h_data.edge_types},
                input_nodes=('net', mask),
                batch_size=6400 
            )

In [139]:
for data in loader:
    break

In [140]:
data

HeteroData(
  batch=[708535],
  num_vn=10093,
  node={
    x=[0, 19],
    y=[0],
    n_id=[0],
  },
  net={
    x=[1280, 2],
    y=[1280],
    n_id=[1280],
    input_id=[1280],
    batch_size=1280,
  },
  (node, as_a_sink_of, net)={
    edge_index=[2, 0],
    e_id=[0],
  },
  (node, as_a_source_of, net)={
    edge_index=[2, 0],
    e_id=[0],
  },
  (net, connected_to, node)={
    edge_index=[2, 0],
    e_id=[0],
  }
)

In [125]:
data.batch = data.batch[data['node'].n_id]

In [42]:
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

hetero_data = OGB_MAG("test")[0]

loader = NeighborLoader(
    hetero_data,
    num_neighbors={key: [30] * 2 for key in hetero_data.edge_types},
    batch_size=128,
    input_nodes=('paper', hetero_data['paper'].train_mask),
)

In [48]:
hetero_data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389],
  },
  author={ num_nodes=1134649 },
  institution={ num_nodes=8740 },
  field_of_study={ num_nodes=59965 },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 5416271] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] }
)

In [103]:
mask = torch.tensor([True for idx in range(len(h_data['net'].x))])

In [50]:
import os
import shutil

In [51]:
current_dir = "cross_design_data"

In [53]:
# Loop through all items in the current directory
for folder in os.listdir(current_dir):
    folder_path = os.path.join(current_dir, folder)
    
    # Check if the item is a directory
    if os.path.isdir(folder_path):
        # Check if "net2sink_nodes.pkl" exists in the directory
        if not os.path.exists(os.path.join(folder_path, "net2sink_nodes.pkl")):
            # If not, remove the directory
            print(f"Removing directory: {folder_path}")
            shutil.rmtree(folder_path)

Removing directory: cross_design_data/Design_67_vivado_AltSpreadLogic_medium_data
Removing directory: cross_design_data/Design_32_vivado_AltSpreadLogic_low_data
Removing directory: cross_design_data/Design_42_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_data/Design_70_vivado_AltSpreadLogic_low_data
Removing directory: cross_design_data/Design_6_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_data/Design_222_vivado_AltSpreadLogic_medium_data
Removing directory: cross_design_data/Design_32_vivado_AltSpreadLogic_medium_data
Removing directory: cross_design_data/Design_81_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_data/Design_26_vivado_AltSpreadLogic_medium_data
Removing directory: cross_design_data/Design_91_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_data/Design_57_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_data/Design_65_vivado_AltSpreadLogic_high_data
Removing directory: cross_design_d